In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime
import pytz

In [2]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/schedule'

start = '?startDate=2023-10-16'

end = '&endDate=2023-10-18'

In [3]:
skedURL = baseURL + start + end

sked = requests.get(skedURL).json()

In [4]:
sked

{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2023. All Rights Reserved.',
 'totalItems': 16,
 'totalEvents': 0,
 'totalGames': 16,
 'totalMatches': 0,
 'metaData': {'timeStamp': '20231017_152628'},
 'wait': 10,
 'dates': [{'date': '2023-10-16',
   'totalItems': 5,
   'totalEvents': 0,
   'totalGames': 5,
   'totalMatches': 0,
   'games': [{'gamePk': 2023020035,
     'link': '/api/v1/game/2023020035/feed/live',
     'gameType': 'R',
     'season': '20232024',
     'gameDate': '2023-10-16T23:00:00Z',
     'status': {'abstractGameState': 'Final',
      'codedGameState': '7',
      'detailedState': 'Final',
      'statusCode': '7',
      'startTimeTBD': False},
     'teams': {'away': {'leagueRecord': {'wins': 2,
        'losses': 1,
        'ot': 0,
        'type': 'league'},
       'score': 4,
       'team': {'id': 17,
        'name': 'Detroit Red Wings',
        'link

In [5]:
days = sked['dates']

pprint(days)

[{'date': '2023-10-16',
  'events': [],
  'games': [{'content': {'link': '/api/v1/game/2023020035/content'},
             'gameDate': '2023-10-16T23:00:00Z',
             'gamePk': 2023020035,
             'gameType': 'R',
             'link': '/api/v1/game/2023020035/feed/live',
             'season': '20232024',
             'status': {'abstractGameState': 'Final',
                        'codedGameState': '7',
                        'detailedState': 'Final',
                        'startTimeTBD': False,
                        'statusCode': '7'},
             'teams': {'away': {'leagueRecord': {'losses': 1,
                                                 'ot': 0,
                                                 'type': 'league',
                                                 'wins': 2},
                                'score': 4,
                                'team': {'id': 17,
                                         'link': '/api/v1/teams/17',
                              

In [6]:
eastern = pytz.timezone('US/Eastern')

for day in days:
    date = day['date']
    games = day['games']
    
    day_of_week = datetime.strptime(date, '%Y-%m-%d').strftime('%A').upper()
    print(day_of_week)
    
    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']
        venue = game['venue']['name']
        game_time_utc = datetime.fromisoformat(game['gameDate'][:-1]).replace(tzinfo=pytz.utc)
        game_time_et = game_time_utc.astimezone(eastern)
        
        formatted_time = game_time_et.strftime('%I:%M %p').lstrip('0').replace(':00', '').replace('PM', 'p.m.')
        
#         print(f"On {date}, {away_team} at {home_team}")
        print(f"{away_team} at {home_team}")
        print(f"{formatted_time}, {venue}")
        print()
        

MONDAY
Detroit Red Wings at Columbus Blue Jackets
7 p.m., Nationwide Arena

Florida Panthers at New Jersey Devils
7 p.m., Prudential Center

Arizona Coyotes at New York Rangers
7 p.m., Madison Square Garden

Chicago Blackhawks at Toronto Maple Leafs
7 p.m., Scotiabank Arena

Calgary Flames at Washington Capitals
7 p.m., Capital One Arena

TUESDAY
Vancouver Canucks at Philadelphia Flyers
6 p.m., Wells Fargo Center

Minnesota Wild at Montréal Canadiens
7 p.m., Centre Bell

Tampa Bay Lightning at Buffalo Sabres
7:30 p.m., KeyBank Center

Arizona Coyotes at New York Islanders
7:30 p.m., UBS Arena

Edmonton Oilers at Nashville Predators
8 p.m., Bridgestone Arena

Los Angeles Kings at Winnipeg Jets
8 p.m., Canada Life Centre

Colorado Avalanche at Seattle Kraken
10 p.m., Climate Pledge Arena

Carolina Hurricanes at San Jose Sharks
10:30 p.m., SAP Center at San Jose

Dallas Stars at Vegas Golden Knights
10:30 p.m., T-Mobile Arena

WEDNESDAY
Washington Capitals at Ottawa Senators
7 p.m., Canad

In [7]:
teams_data = {}

for day in days:
    games = day['games']
    
    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']
        
        if home_team not in teams_data:
            teams_data[home_team] = {'played': 0, 'home': 0, 'away': 0}
        if away_team not in teams_data:
            teams_data[away_team] = {'played': 0, 'home': 0, 'away': 0}
        
        teams_data[home_team]['played'] += 1
        teams_data[away_team]['played'] += 1
        teams_data[home_team]['home'] += 1
        teams_data[away_team]['away'] += 1

# Create a DataFrame
teams_df = pd.DataFrame.from_dict(teams_data, orient='index')
teams_df = teams_df.sort_values(by=['played', 'home'], ascending=False)

print(teams_df)

                       played  home  away
Detroit Red Wings           2     1     1
Washington Capitals         2     1     1
Arizona Coyotes             2     0     2
Columbus Blue Jackets       1     1     0
New Jersey Devils           1     1     0
New York Rangers            1     1     0
Toronto Maple Leafs         1     1     0
Philadelphia Flyers         1     1     0
Montréal Canadiens          1     1     0
Buffalo Sabres              1     1     0
New York Islanders          1     1     0
Nashville Predators         1     1     0
Winnipeg Jets               1     1     0
Seattle Kraken              1     1     0
San Jose Sharks             1     1     0
Vegas Golden Knights        1     1     0
Ottawa Senators             1     1     0
Florida Panthers            1     0     1
Chicago Blackhawks          1     0     1
Calgary Flames              1     0     1
Vancouver Canucks           1     0     1
Minnesota Wild              1     0     1
Tampa Bay Lightning         1     

In [8]:
def check_consecutive_dates(game1, game2):
    date_format = '%Y-%m-%dT%H:%M:%SZ'
    date1 = datetime.strptime(game1['gameDate'], date_format)
    date2 = datetime.strptime(game2['gameDate'], date_format)
    return (date2 - date1).days == 1

teams_playing_consecutive_days = set()

for i in range(len(days)-1):
    current_day_games = days[i]['games']
    next_day_games = days[i+1]['games']
    
    teams_on_next_day = set()
    
    for game2 in next_day_games:
        home_team_2 = game2['teams']['home']['team']['name']
        away_team_2 = game2['teams']['away']['team']['name']
        teams_on_next_day.add(home_team_2)
        teams_on_next_day.add(away_team_2)

    for game1 in current_day_games:
        home_team_1 = game1['teams']['home']['team']['name']
        away_team_1 = game1['teams']['away']['team']['name']
        
        if (home_team_1 in teams_on_next_day) and check_consecutive_dates(game1, game2):
            teams_playing_consecutive_days.add(home_team_1)
        elif (away_team_1 in teams_on_next_day) and check_consecutive_dates(game1, game2):
            teams_playing_consecutive_days.add(away_team_1)

print("Teams playing on consecutive days:")
for team in teams_playing_consecutive_days:
    print(team)

Teams playing on consecutive days:
Arizona Coyotes


In [10]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/game/'
appendix = '/feed/live'

year = '202302'
game = 1

awaylogs = {}
homelogs = {}


for i in range(39):
    
    gameURL = baseURL + year + str(i+1).zfill(4) + appendix
    
    print(gameURL)

    game = requests.get(gameURL).json()
    
    away = game['liveData']['boxscore']['teams']['away']['players']
    awayname = game['gameData']['teams']['away']['abbreviation']
    home = game['liveData']['boxscore']['teams']['home']['players']
    homename = game['gameData']['teams']['home']['abbreviation']
    birthday = game['gameData']['players']

    if game['liveData']['linescore']['currentPeriod'] > 3:
        OT = True
    else:
        OT = False

    awayteam = []

    date = game['gameData']['datetime']['dateTime']

    datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

    utc_timestamp = pytz.utc.localize(datetime_obj)
    eastern_timezone = pytz.timezone('US/Eastern')
    eastern_time = utc_timestamp.astimezone(eastern_timezone)

    formatted_date_string = eastern_time.strftime('%d-%m-%Y')

    for player in away:
        awayteam.append(player)

    for player in awayteam:
        name = away[player]['person']['fullName']
        birth_date = birthday[player]['birthDate']
        stats = away[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = away[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = away[player]['position']['abbreviation']
            stats['name'] = name

        awaylogs[player  + '-' + formatted_date_string] = stats
        awaylogs[player  + '-' + formatted_date_string]['position'] = position
        awaylogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        awaylogs[player  + '-' + formatted_date_string]['name'] = name
        awaylogs[player  + '-' + formatted_date_string]['team'] = awayname
        awaylogs[player  + '-' + formatted_date_string]['opponent'] = homename
        awaylogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date


    hometeam = []

    # date = game['gameData']['datetime']['dateTime']

    for player in home:
        hometeam.append(player)

    for player in hometeam:
        name = home[player]['person']['fullName']
        birth_date = birthday[player]['birthDate']
        stats = home[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = home[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = home[player]['position']['abbreviation']
            stats['name'] = name

        homelogs[player  + '-' + formatted_date_string] = stats
        homelogs[player  + '-' + formatted_date_string]['position'] = position
        homelogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        homelogs[player  + '-' + formatted_date_string]['name'] = name
        homelogs[player  + '-' + formatted_date_string]['team'] = homename
        homelogs[player  + '-' + formatted_date_string]['opponent'] = awayname
        homelogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        
#         game+=1

https://statsapi.web.nhl.com/api/v1/game/2023020001/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020002/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020003/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020004/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020005/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020006/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020007/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020008/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020009/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020010/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020011/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020012/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020013/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020014/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020015/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020016/feed/live
https://

In [11]:
home_df = pd.DataFrame(homelogs)
home_df = home_df.transpose()

columns_to_convert1 = ['timeOnIce', 'assists', 'goals', 'shots', 'hits', 'powerPlayGoals', 'powerPlayAssists', 'penaltyMinutes']
columns_to_convert2 = ['faceOffWins', 'faceoffTaken', 'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists']
columns_to_convert3 = ['blocked', 'plusMinus', 'evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'pim', 'shutout']
columns_to_convert4 = ['saves', 'powerPlaySaves', 'shortHandedSaves', 'evenSaves', 'shortHandedShotsAgainst', 'evenShotsAgainst', 'powerPlayShotsAgainst']

home_df[columns_to_convert1] = home_df[columns_to_convert1].fillna(0).astype(int)
home_df[columns_to_convert2] = home_df[columns_to_convert2].fillna(0).astype(int)
home_df[columns_to_convert3] = home_df[columns_to_convert3].fillna(0).astype(int)
home_df[columns_to_convert4] = home_df[columns_to_convert4].fillna(0).astype(int)
# , errors='coerce'

home_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 831 entries, ID8478519-10-10-2023 to ID8481656-16-10-2023
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   timeOnIce                   831 non-null    int32 
 1   assists                     831 non-null    int32 
 2   goals                       831 non-null    int32 
 3   shots                       831 non-null    int32 
 4   hits                        831 non-null    int32 
 5   powerPlayGoals              831 non-null    int32 
 6   powerPlayAssists            831 non-null    int32 
 7   penaltyMinutes              831 non-null    int32 
 8   faceOffPct                  273 non-null    object
 9   faceOffWins                 831 non-null    int32 
 10  faceoffTaken                831 non-null    int32 
 11  takeaways                   831 non-null    int32 
 12  giveaways                   831 non-null    int32 
 13  shortHandedGoals   

In [12]:
away_df = pd.DataFrame(awaylogs)
away_df = away_df.transpose()

# columns_to_convert1 = ['timeOnIce', 'assists', 'goals', 'shots', 'hits', 'powerPlayGoals', 'powerPlayAssists', 'penaltyMinutes']
# columns_to_convert2 = ['faceOffWins', 'faceoffTaken', 'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists']
# columns_to_convert3 = ['blocked', 'plusMinus', 'evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'pim']
# columns_to_convert4 = ['saves', 'powerPlaySaves', 'shortHandedSaves', 'evenSaves', 'shortHandedShotsAgainst', 'evenShotsAgainst', 'powerPlayShotsAgainst']

away_df[columns_to_convert1] = away_df[columns_to_convert1].fillna(0).astype(int)
away_df[columns_to_convert2] = away_df[columns_to_convert2].fillna(0).astype(int)
away_df[columns_to_convert3] = away_df[columns_to_convert3].fillna(0).astype(int)
away_df[columns_to_convert4] = away_df[columns_to_convert4].fillna(0).astype(int)
# , errors='coerce'

away_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 836 entries, ID8482062-10-10-2023 to ID8482679-16-10-2023
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   timeOnIce                   836 non-null    int32 
 1   assists                     836 non-null    int32 
 2   goals                       836 non-null    int32 
 3   shots                       836 non-null    int32 
 4   hits                        836 non-null    int32 
 5   powerPlayGoals              836 non-null    int32 
 6   powerPlayAssists            836 non-null    int32 
 7   penaltyMinutes              836 non-null    int32 
 8   faceOffWins                 836 non-null    int32 
 9   faceoffTaken                836 non-null    int32 
 10  takeaways                   836 non-null    int32 
 11  giveaways                   836 non-null    int32 
 12  shortHandedGoals            836 non-null    int32 
 13  shortHandedAssists 

In [13]:
ID8476887 = away_df.loc[away_df['name'] == 'Filip Forsberg']
ID8476887

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,...,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage,shutout,shortHandedSavePercentage
ID8476887-10-10-2023,1251,2,0,6,1,0,0,0,0,0,...,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN
ID8476887-14-10-2023,1292,1,0,3,2,0,0,0,0,0,...,0,0,0,0,NaN,NaN,NaN,NaN,0,NaN


In [14]:
all_df = pd.concat([home_df, away_df], axis=0)
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1667 entries, ID8478519-10-10-2023 to ID8482679-16-10-2023
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   timeOnIce                   1667 non-null   int32 
 1   assists                     1667 non-null   int32 
 2   goals                       1667 non-null   int32 
 3   shots                       1667 non-null   int32 
 4   hits                        1667 non-null   int32 
 5   powerPlayGoals              1667 non-null   int32 
 6   powerPlayAssists            1667 non-null   int32 
 7   penaltyMinutes              1667 non-null   int32 
 8   faceOffPct                  550 non-null    object
 9   faceOffWins                 1667 non-null   int32 
 10  faceoffTaken                1667 non-null   int32 
 11  takeaways                   1667 non-null   int32 
 12  giveaways                   1667 non-null   int32 
 13  shortHandedGoals  

In [15]:
all_df['date'] = pd.to_datetime(all_df['date'], format='%d-%m-%Y')
all_df['birthdate'] = pd.to_datetime(all_df['birthdate'], format='%Y-%m-%d')
player_list = all_df['name'].unique()
all_df['name'].nunique()
all_df = all_df.sort_values(by='date')

In [16]:
goalies = all_df.loc[all_df['position'] == "G"]
goalies

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,...,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout
ID8475852-10-10-2023,3600,0,0,41,0,0,0,0,NaN,0,...,35,0,37,4,W,95.121951,100.0,NaN,94.594595,0
ID8477424-10-10-2023,3436,0,0,33,0,0,0,0,NaN,0,...,21,0,23,10,L,87.878788,80.0,NaN,91.304348,0
ID8475831-10-10-2023,3577,0,0,27,0,0,0,0,NaN,0,...,21,0,24,3,L,88.888889,100.0,NaN,87.5,0
ID8477992-10-10-2023,3600,0,0,31,0,0,0,0,NaN,0,...,21,1,23,7,W,90.322581,85.714286,100.0,91.304348,0
ID8477465-10-10-2023,3450,0,0,35,0,0,0,0,NaN,0,...,26,1,29,5,L,91.428571,100.0,100.0,89.655172,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8479361-16-10-2023,3373,0,0,30,0,0,0,0,NaN,0,...,24,0,26,4,L,90.0,75.0,NaN,92.307692,0
ID8475311-16-10-2023,3900,0,0,40,0,0,0,0,NaN,0,...,31,1,32,7,W,95.0,85.714286,100.0,96.875,0
ID8477484-16-10-2023,3556,0,0,28,0,0,0,0,NaN,0,...,18,2,20,6,L,85.714286,66.666667,100.0,90.0,0
ID8478048-16-10-2023,3595,0,0,27,0,0,0,0,NaN,0,...,19,0,19,8,W,96.296296,87.5,NaN,100.0,0


In [17]:
for index, row in all_df.iterrows():
    d = 0
    so = 0
    if row['position'] == "G":
        if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
            all_df.at[index, 'shutout'] = 0
        if row['shutout'] == 1:
            so = 3
        else:
            so = 0
        if row['decision'] == "W":
            d = 4
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
        all_df.at[index, 'shots_against'] = shots_against
        ga = (shots_against - row['saves']) * -2
        all_df.at[index, 'goals_against'] = abs(ga)/2
        saves = row['saves'] * .2
        fp = d + so + ga + saves
        all_df.at[index, 'fantasyPoints'] = fp
#         row['fantasyPoints'] = fp
    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
        all_df.at[index, 'fantasyPoints'] = fp
#         row['fantasyPoints'] = fp

In [18]:
all_df['fantasyPoints']

ID8478519-10-10-2023    0.1
ID8474870-10-10-2023    0.8
ID8476960-10-10-2023    1.1
ID8479458-10-10-2023    0.0
ID8470621-10-10-2023    2.3
                       ... 
ID8475287-16-10-2023    4.2
ID8476525-16-10-2023    0.0
ID8477931-16-10-2023    0.0
ID8476292-16-10-2023    0.9
ID8482679-16-10-2023    0.3
Name: fantasyPoints, Length: 1667, dtype: float64

In [19]:
skaters = all_df.loc[all_df['position'] != "G"]
skaters

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,...,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,fantasyPoints,shots_against,goals_against
ID8478519-10-10-2023,1002,0,0,1,0,0,0,0,60.0,9,...,0,NaN,NaN,NaN,NaN,NaN,0,0.1,NaN,NaN
ID8474870-10-10-2023,983,0,0,3,0,0,0,0,20.0,1,...,0,NaN,NaN,NaN,NaN,NaN,0,0.8,NaN,NaN
ID8476960-10-10-2023,610,1,0,1,0,0,0,0,NaN,0,...,0,NaN,NaN,NaN,NaN,NaN,0,1.1,NaN,NaN
ID8479458-10-10-2023,0,0,0,0,0,0,0,0,NaN,0,...,0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
ID8470621-10-10-2023,1008,2,0,2,1,0,0,0,NaN,0,...,0,NaN,NaN,NaN,NaN,NaN,0,2.3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8475287-16-10-2023,867,1,1,6,1,1,0,2,42.86,6,...,0,NaN,NaN,NaN,NaN,NaN,0,4.2,NaN,NaN
ID8476525-16-10-2023,0,0,0,0,0,0,0,0,NaN,0,...,0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
ID8477931-16-10-2023,0,0,0,0,0,0,0,0,NaN,0,...,0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
ID8476292-16-10-2023,955,0,0,1,3,0,0,2,NaN,0,...,0,NaN,NaN,NaN,NaN,NaN,0,0.9,NaN,NaN


In [20]:
summary_stats = all_df.groupby('name').agg({
    'team': 'first',
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'fantasyPoints': 'sum'
    # Add more columns as needed
})

summary_stats['FPP60'] = (summary_stats['fantasyPoints'] / summary_stats['timeOnIce'] * 3600).round(2)

summary_stats = summary_stats.sort_values(by='FPP60', ascending=False)
# pd.set_option('display.max_rows', None)
summary_stats.head(100)

,team,timeOnIce,assists,goals,shots,fantasyPoints,FPP60
name,,,,,,,
Brock Boeser,VAN,2106,1,4,9,10.4,17.78
Jack Studnicka,VAN,439,0,1,1,2.1,17.22
MacKenzie Entwistle,CHI,498,0,1,1,2.3,16.63
Elias Pettersson,VAN,2288,5,1,6,10.5,16.52
Nils Hoglander,VAN,1268,2,1,1,5.5,15.62
...,...,...,...,...,...,...,...
Charlie McAvoy,BOS,2945,2,0,3,6.7,8.19
Jake Evans,MTL,1671,0,1,3,3.8,8.19
Tyler Johnson,CHI,3787,0,3,9,8.6,8.18


In [21]:
opponent_stats = skaters.groupby('opponent').agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'date': 'nunique'
    # Add more columns as needed
})

opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['date'], 2)

opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=True, method='min').astype(int)

opponent_stats

,timeOnIce,assists,goals,shots,fantasyPoints,date,FPAPG,FPAPG_Rank
opponent,,,,,,,,
ANA,35256,14,7,73,55.7,2,27.85,21
ARI,35575,10,5,64,48.7,2,24.35,15
BOS,35372,6,3,56,45.0,2,22.50,10
BUF,35385,13,8,60,61.4,2,30.70,26
CAR,53099,21,14,85,95.6,3,31.87,28
CBJ,53009,18,11,107,79.0,3,26.33,18
CGY,53782,16,10,89,78.3,3,26.10,17
CHI,70219,15,9,145,91.3,4,22.82,11
COL,36321,4,3,57,39.3,2,19.65,5


In [22]:
opponent_stats_by_pos = skaters.groupby(['opponent', 'position']).agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))]
    # Add more columns as needed
})
opponent_stats_by_pos

timeOnIce assists goals shots fantasyPoints      
                        sum     sum   sum   sum           sum  mean
opponent position                                                  
ANA      C            11817       5     3    26          19.5  1.62
         D            13497       8     1    26          22.6  1.88
         LW            4288       1     2    10           8.3  1.66
         N/A              0       0     0     0           0.0  0.00
         RW            5654       0     1    11           5.3  0.76
...                     ...     ...   ...   ...           ...   ...
WSH      C            13304       5     5    35          26.0  1.86
         D            14164       3     0    22          19.0  1.58
         LW            4224       2     0     4           4.7  0.94
         N/A              0       0     0     0           0.0  0.00
         RW            4502       1     1    14           5.8  1.16

[160 rows x 6 columns]

In [23]:
teamURL = 'https://statsapi.web.nhl.com/api/v1/teams'

In [24]:
teams = requests.get(teamURL).json()
teams = teams['teams']

In [25]:


team_names = pd.DataFrame()

for team in teams:
    abbreviation = team['abbreviation']
    name = team['name']
    
    temp = pd.DataFrame({'abbreviation': [abbreviation], 'name': [name]})
    
    team_names = pd.concat([team_names, temp], ignore_index=True)
    
team_names

,abbreviation,name
0,NJD,New Jersey Devils
1,NYI,New York Islanders
2,NYR,New York Rangers
3,PHI,Philadelphia Flyers
4,PIT,Pittsburgh Penguins
5,BOS,Boston Bruins
6,BUF,Buffalo Sabres
7,MTL,Montréal Canadiens
8,OTT,Ottawa Senators
9,TOR,Toronto Maple Leafs


In [26]:
opponent_stats = opponent_stats.rename_axis('abbreviation')
opponent_stats

,timeOnIce,assists,goals,shots,fantasyPoints,date,FPAPG,FPAPG_Rank
abbreviation,,,,,,,,
ANA,35256,14,7,73,55.7,2,27.85,21
ARI,35575,10,5,64,48.7,2,24.35,15
BOS,35372,6,3,56,45.0,2,22.50,10
BUF,35385,13,8,60,61.4,2,30.70,26
CAR,53099,21,14,85,95.6,3,31.87,28
CBJ,53009,18,11,107,79.0,3,26.33,18
CGY,53782,16,10,89,78.3,3,26.10,17
CHI,70219,15,9,145,91.3,4,22.82,11
COL,36321,4,3,57,39.3,2,19.65,5


In [27]:
merged_df = pd.merge(opponent_stats, team_names, on='abbreviation')
merged_df

,abbreviation,timeOnIce,assists,goals,shots,fantasyPoints,date,FPAPG,FPAPG_Rank,name
0,ANA,35256,14,7,73,55.7,2,27.85,21,Anaheim Ducks
1,ARI,35575,10,5,64,48.7,2,24.35,15,Arizona Coyotes
2,BOS,35372,6,3,56,45.0,2,22.50,10,Boston Bruins
3,BUF,35385,13,8,60,61.4,2,30.70,26,Buffalo Sabres
4,CAR,53099,21,14,85,95.6,3,31.87,28,Carolina Hurricanes
5,CBJ,53009,18,11,107,79.0,3,26.33,18,Columbus Blue Jackets
6,CGY,53782,16,10,89,78.3,3,26.10,17,Calgary Flames
7,CHI,70219,15,9,145,91.3,4,22.82,11,Chicago Blackhawks
8,COL,36321,4,3,57,39.3,2,19.65,5,Colorado Avalanche
9,DAL,18550,2,1,24,17.4,1,17.40,3,Dallas Stars


In [28]:
# eastern = pytz.timezone('US/Eastern')

# for day in days:
#     date = day['date']
#     games = day['games']
    
#     day_of_week = datetime.strptime(date, '%Y-%m-%d').strftime('%A').upper()
#     print(day_of_week)
    
#     for game in games:
#         home_team = game['teams']['home']['team']['name']
#         away_team = game['teams']['away']['team']['name']
#         venue = game['venue']['name']
#         game_time_utc = datetime.fromisoformat(game['gameDate'][:-1]).replace(tzinfo=pytz.utc)
#         game_time_et = game_time_utc.astimezone(eastern)
        
#         formatted_time = game_time_et.strftime('%I:%M %p').lstrip('0').replace(':00', '').replace('PM', 'p.m.')
        
#         home_abbr = merged_df.loc[merged_df['name'] == home_team, 'abbreviation'].values[0]
#         home_top5 = top_players_by_team.loc[top_players_by_team['team'] == home_abbr]
        
#         away_abbr = merged_df.loc[merged_df['name'] == away_team, 'abbreviation'].values[0]
#         away_top5 = top_players_by_team.loc[top_players_by_team['team'] == away_abbr]
        
# #         print(f"On {date}, {away_team} at {home_team}")
#         print(f"\033[1m{away_team} at {home_team}\033[0m")
#         print(f"{formatted_time}, {venue}")
#         print()
#         print(f"{away_abbr}: {merged_df.loc[merged_df['name'] == away_team, 'FPAPG'].values[0]} FPAPG, Rank: {merged_df.loc[merged_df['name'] == away_team, 'FPAPG_Rank'].values[0]}")
#         for index, row in away_top5.iterrows():
#             print(f"{index}, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
#         print()
#         print(f"{home_abbr}: {merged_df.loc[merged_df['name'] == home_team, 'FPAPG'].values[0]} FPAPG, Rank: {merged_df.loc[merged_df['name'] == home_team, 'FPAPG_Rank'].values[0]}")
#         for index, row in home_top5.iterrows():
#             print(f"{index}, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
        
#         print()
# #         print("\033[1mThis is bold text\033[0m")
        

In [29]:
summary_stats

,team,timeOnIce,assists,goals,shots,fantasyPoints,FPP60
name,,,,,,,
Brock Boeser,VAN,2106,1,4,9,10.4,17.78
Jack Studnicka,VAN,439,0,1,1,2.1,17.22
MacKenzie Entwistle,CHI,498,0,1,1,2.3,16.63
Elias Pettersson,VAN,2288,5,1,6,10.5,16.52
Nils Hoglander,VAN,1268,2,1,1,5.5,15.62
...,...,...,...,...,...,...,...
Ukko-Pekka Luukkonen,BUF,0,0,0,0,0.0,NaN
Urho Vaakanainen,ANA,0,0,0,0,0.0,NaN
Viktor Arvidsson,LAK,0,0,0,0,0.0,NaN


In [30]:
top_players_by_team = summary_stats.groupby('team').apply(lambda x: x.nlargest(5, 'FPP60'))

In [31]:
top_players_by_team.reset_index(level='team', drop=True, inplace=True)

In [32]:
top_players_by_team.loc[top_players_by_team['team'] == 'ANA']

,team,timeOnIce,assists,goals,shots,fantasyPoints,FPP60
name,,,,,,,
Frank Vatrano,ANA,2491,0,3,8,9.1,13.15
Pavel Mintyukov,ANA,2169,0,1,4,5.2,8.63
Mason McTavish,ANA,2193,2,1,5,5.2,8.54
Cam Fowler,ANA,2904,1,1,2,6.7,8.31
Sam Carrick,ANA,975,0,1,1,2.2,8.12


In [33]:
top_players_by_team

,team,timeOnIce,assists,goals,shots,fantasyPoints,FPP60
name,,,,,,,
Frank Vatrano,ANA,2491,0,3,8,9.1,13.15
Pavel Mintyukov,ANA,2169,0,1,4,5.2,8.63
Mason McTavish,ANA,2193,2,1,5,5.2,8.54
Cam Fowler,ANA,2904,1,1,2,6.7,8.31
Sam Carrick,ANA,975,0,1,1,2.2,8.12
...,...,...,...,...,...,...,...
Matthew Phillips,WSH,1444,1,1,3,3.8,9.47
Nic Dowd,WSH,1745,0,0,0,3.5,7.22
Darcy Kuemper,WSH,3900,0,0,40,7.6,7.02


In [34]:
eastern = pytz.timezone('US/Eastern')

for day in days:
    date = day['date']
    games = day['games']
    
    day_of_week = datetime.strptime(date, '%Y-%m-%d').strftime('%A').upper()
    print(day_of_week)
    
    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']
        venue = game['venue']['name']
        game_time_utc = datetime.fromisoformat(game['gameDate'][:-1]).replace(tzinfo=pytz.utc)
        game_time_et = game_time_utc.astimezone(eastern)
        
        formatted_time = game_time_et.strftime('%I:%M %p').lstrip('0').replace(':00', '').replace('PM', 'p.m.')
        
        home_abbr = merged_df.loc[merged_df['name'] == home_team, 'abbreviation'].values[0]
        home_top5 = top_players_by_team.loc[top_players_by_team['team'] == home_abbr]
        
        away_abbr = merged_df.loc[merged_df['name'] == away_team, 'abbreviation'].values[0]
        away_top5 = top_players_by_team.loc[top_players_by_team['team'] == away_abbr]
        
#         print(f"On {date}, {away_team} at {home_team}")
        print(f"\033[1m{away_team} at {home_team}\033[0m")
        print(f"{formatted_time}, {venue}")
        print()
        print(f"{away_abbr}: {merged_df.loc[merged_df['name'] == away_team, 'FPAPG'].values[0]} FPAPG, Rank: {merged_df.loc[merged_df['name'] == away_team, 'FPAPG_Rank'].values[0]}")
        print(f"\033[1mTop Players in FPP60\033[0m")
        for index, row in away_top5.iterrows():
            print(f"{index}, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
        print()
        print(f"{home_abbr}: {merged_df.loc[merged_df['name'] == home_team, 'FPAPG'].values[0]} FPAPG, Rank: {merged_df.loc[merged_df['name'] == home_team, 'FPAPG_Rank'].values[0]}")
        print(f"\033[1mTop Players in FPP60\033[0m")
        for index, row in home_top5.iterrows():
            print(f"{index}, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
        
        print()
#         print("\033[1mThis is bold text\033[0m")
        

MONDAY
Detroit Red Wings at Columbus Blue Jackets
7 p.m., Nationwide Arena

DET: 20.37 FPAPG, Rank: 6
Top Players in FPP60
Alex DeBrincat, FPP60: 12.82, FP: 11.0
James Reimer, FPP60: 11.6, FP: 11.6
Robby Fabbri, FPP60: 11.4, FP: 2.8
Moritz Seider, FPP60: 8.83, FP: 10.5
Jake Walman, FPP60: 8.66, FP: 8.7

CBJ: 26.33 FPAPG, Rank: 18
Top Players in FPP60
Boone Jenner, FPP60: 11.08, FP: 10.7
Patrik Laine, FPP60: 8.3, FP: 6.8
Justin Danforth, FPP60: 7.28, FP: 4.6
Kirill Marchenko, FPP60: 7.06, FP: 5.6
Jake Bean, FPP60: 6.86, FP: 6.6

Florida Panthers at New Jersey Devils
7 p.m., Prudential Center

FLA: 29.7 FPAPG, Rank: 25
Top Players in FPP60
Sam Reinhart, FPP60: 10.53, FP: 11.1
Evan Rodrigues, FPP60: 9.93, FP: 9.1
Matthew Tkachuk, FPP60: 7.49, FP: 7.3
Niko Mikkola, FPP60: 6.57, FP: 5.8
Carter Verhaeghe, FPP60: 6.1, FP: 5.6

NJD: 28.6 FPAPG, Rank: 22
Top Players in FPP60
Jesper Bratt, FPP60: 12.88, FP: 12.7
Erik Haula, FPP60: 10.38, FP: 7.3
Jack Hughes, FPP60: 10.32, FP: 11.6
Dougie Hamilto

In [35]:
team_check = merged_df['name'].values

teams_df.index.values

for team in team_check:
    if team not in teams_df.index.values:
        print(team)

Anaheim Ducks
Boston Bruins
St. Louis Blues


In [37]:
summary_stats.loc[summary_stats['team'] == 'VAN']

,team,timeOnIce,assists,goals,shots,fantasyPoints,FPP60
name,,,,,,,
Brock Boeser,VAN,2106,1,4,9,10.4,17.78
Jack Studnicka,VAN,439,0,1,1,2.1,17.22
Elias Pettersson,VAN,2288,5,1,6,10.5,16.52
Nils Hoglander,VAN,1268,2,1,1,5.5,15.62
J.T. Miller,VAN,2571,3,1,4,8.5,11.90
Quinn Hughes,VAN,2830,4,0,4,7.9,10.05
Andrei Kuzmenko,VAN,1707,1,1,4,4.4,9.28
Conor Garland,VAN,1174,0,1,4,2.9,8.89
Sam Lafferty,VAN,1960,1,1,2,4.3,7.90
